<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/120_%E5%9E%8B%E3%83%92%E3%83%B3%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

型ヒント
========

アノテーション
--------------

**アノテーション**（annotation）とは、グローバル変数、クラス変数、関数の引数や戻り値に、注釈として埋め込む情報（メタデータ）のことである。特殊な記法を用いて埋め込まれた、グローバル変数、クラス変数、関数のアノテーションは、式として評価され、その値がそれぞれモジュール、クラス、関数の `__annotations__` 特殊属性に保持される。

アノテーションの記法を示す:

In [ ]:
# グローバル変数アノテーションの例
global_var1: "変数に関する注釈" = True
global_var2: "代入を後回しにしてもよい"

# クラス変数アノテーションの例
class C:
    field1: "クラス変数に関する注釈" = True
    field2: "代入を後回しにしてもよい"

# 関数アノテーションの例
def f(x: "xに関する注釈", y: "yに関する注釈" = 1) -> "戻り値に関する注釈":
    pass

print(f"{__annotations__=}")
print(f"{C.__annotations__=}")
print(f"{f.__annotations__=}")
__annotations__.clear()

__annotations__={'global_var1': '変数に関する注釈', 'global_var2': '代入を後回しにしてもよい'}
C.__annotations__={'field1': 'クラス変数に関する注釈', 'field2': '代入を後回しにしてもよい'}
f.__annotations__={'x': 'xに関する注釈', 'y': 'yに関する注釈', 'return': '戻り値に関する注釈'}


上記のコードのように、グローバル変数とクラス変数のアノテーションは、代入と分離して書くことができる。先にアノテーションだけ書いておく形式は、変数の宣言となる。宣言された変数は、値が代入されるまではまだ定義されてなく、ローカルな名前空間に変数は存在しない。

In [ ]:
class C:
    x: "注釈"

    def __init__(self):
        self.x = 0

assert "x" not in C.__dict__
# print(C.x)  # 参照しようとするとエラーが発生する

class D:
    x: "注釈" = 0

assert "x" in D.__dict__

PEP 8 によれば、変数とアノテーションの間のコロン `:` の前にはスペースを入れず、`->` の両側には常にスペースを入れなければならない。デフォルト値付き引数について `=` の両側にスペースを入れてはいけないのが原則であるが、アノテーションを入れるときは `=` の前後にスペースを入れなければならない。

ソースコード中に注釈を記入する仕組みにはコメントもあるが、これは構文解析の過程で削除され、言語処理系の動作に影響を及ぼさない。一方、アノテーションは、オブジェクトの属性の値として保持されるため、言語処理系に指示を与えることが可能である。ただし、Python 言語のサポートはアノテーション構文の導入にとどまっており、バイトコンパイル時や実行時に参照させるということは全くできていないのが現状である。

型アノテーション
----------------

[PEP 483（The Theory of Type Hints）](https://peps.python.org/pep-0483/)は、言語レベルで用途が定まってないアノテーションを外部ツールによる静的型チェックに利用することを提案するものである。

Python のような動的型付け言語は、型を指定しない分、ソースコードの記述量を減らすことができ、アルゴリズムを書くことに集中しやすいというメリットがあるが、大規模プロジェクトではソースコードの可読性や保守性が悪くなるというデメリットの方が大きくなる。

そこで PEP 483 は、アノテーションに変数や関数の引数、戻り値の期待される型の情報を記述し、外部ツールに参照させて静的型チェックを行わせることを提唱した。この外部ツール（静的型チェッカー）を使うことにより、プログラムの実行前に型の整合性に関するエラーを回避できるようになる。また、エディタの入力支援がアノテーションを参照して入力候補を絞り込むことができるようにもなる。

型の情報を記述するアノテーションを**型アノテーション**と呼ぶ。型アノテーションなどによって静的型チェッカーに型情報を伝える仕組みを**型ヒント**（type hints）という。

一方、型アノテーションが使われるようになっても、Python 言語の処理系がアノテーションを参照しないことに変更はないので、型アノテーションによってバイトコンパイル時に最適化が強化されるということはない。型アノテーションの利用はユーザー側に任されている。

アノテーションは式として評価されるから、型アノテーションは Python 式でなければならない。ただし、あらゆる式を型アノテーションの形式とすると、静的型チェッカーが対応できない。そこで、型アノテーションとして許容される形式は、次の形式に制限される（[PEP 484](https://peps.python.org/pep-0484/)）。

  1. 組み込みクラス（組み込みのコンストラクタが公開されている型）
  2. `collections.abc` モジュールが提供する抽象基底クラス（Python 3.9 で追加）
  3. 標準ライブラリの `types` モジュールで定義された型
  4. ユーザー定義クラス（標準ライブラリまたはサードパーティ製のモジュールで定義されたものを含む）
  5. 型アノテーションで用いるとき `None` という式は `type(None)` または `types.NoneType` と等価であるとみなされる

条件 1 により、以下は型アノテーションで利用できる。

  * 数値の型: `int`、`float`、`complex`
  * ブール値の型: `bool`
  * 文字列・バイト列の型: `str`、`bytes`、`bytearray`
  * コレクションの型: `list`、`tuple`、`set`、`frozenset`、`dict`

In [ ]:
# グローバル変数の型アノテーションの例
global_var: bool

# クラス変数の型アノテーションの例
class MyClass:
    field: str

# 関数の型アノテーションの例
def func(x: float, y: int = 1) -> None:
    pass

print(f"{__annotations__=}")
print(f"{MyClass.__annotations__=}")
print(f"{func.__annotations__=}")

__annotations__={'global_var': <class 'bool'>}
MyClass.__annotations__={'field': <class 'str'>}
func.__annotations__={'x': <class 'float'>, 'y': <class 'int'>, 'return': None}


関数は return 文がないとき暗黙的に `None` を返すことに注意する。

関数内で定義するローカル変数は、アノテーション構文が導入されていないので、型アノテーションを付ける方法がない。代用として、次のような形式のコメントが使われている:

``` python
def f():
    count = 1  # type: int
    ...
```

また、静的型チェッカーにチェックさせたくない箇所に、次のような形式のコメントが使われている:

``` python
a: int = 1
a = "1"  # type: ignore
```

### 添字表記のサポート ###

Python 3.9 からは、型ヒントが添字表記（`[]`）をサポートするようになった。

添字表記でコレクションに格納するオブジェクトの型を指定できる。

  * `list[要素の型]`
  * `tuple[要素の型]`（1要素）、`tuple[要素の型, 要素の型]`（2要素）、`tuple[要素の型, 要素の型, 要素の型]`（3要素）。以下同様
  * `set[要素の型]`
  * `frozenset[要素の型]`
  * `dict[キーの型, 値の型]`

``` python
users: list[str]
size: tuple[float, float, float]
permissions: set[str]
info: dict[str, int]
```

`tuple` については、タプルを構成する要素のすべてについて型を指定する必要がある。要素がすべて同じ型であるときは、カンマと `...` を使って繰り返しを省略できる。

``` python
size: tuple[float, ...]
```

`dict` については、キーの型と値の型を指定しなければならない。

添字表記は、型の入れ子（ネスト）にも対応する。しかし、複雑な型を書くくらいなら、クラスにするほうがわかりやすいと思う。たとえば、 `list[dict[str, list[int]]]` よりも `list[StudentGrades]` のほうがデータ構造は明らかである。**表現しづらい型を書くくらいならクラスにする**ほうがよい（[最近の Python の型ヒントとの付き合い方:Diary over Finite Fields](https://blog.515hikaru.net/entry/2020/10/17/165228) を参照）。

型アノテーションとして許容される形式ならばどんな型にでも添字表記が使えるので、`type[int]` と指定することもでき、これは `int` 型そのものまたはサブクラスを受け取ることを表す。次のコード

``` python
def func(t: type[int]):
    ...
```

このコードでは、`func()` は引数に `bool` を渡すことができる。`bool` は `int` のサブクラスだからである。

`collections.abc` モジュールが提供する抽象基底クラスについては、以下のようになる。

| 型 | 記法 | 使用例または注意事項 |
|:---|:---|:---|
| 呼び出し可能型 | `collections.abc.Callable[引数の型のリスト, 戻り値の型]` | `Callable[[int], str]`（`int` 型の引数を1つ受け取り`str` を返す関数）<br />`Callable[[], str]`（引数なしで `str` を返す関数） |
| ジェネレーター型 | `collections.abc.Generator[YieldType, SendType, ReturnType]` | `YieldType` は返り値の型。`SendType` は `send()` メソッドで送られる値の<br />型。`ReturnType` は return 文に置く値の型。もしジェネレーターが値を返<br />すだけの場合は、`SendType` と `ReturnType` に `None` を設定する |
| イテレーター型 | `collections.abc.Iterator[YieldType]` | `YieldType` は返り値の型。ジェネレーターの指定にも使用できる |
| イテラブル型 | `collections.abc.Iterable[YieldType]` | `YieldType` は返り値の型。ジェネレーターの指定にも使用できる |
| シーケンス型 | `collections.abc.Sequence[要素の型]` | |

### 複数の型の許可 ###

Python 3.9 からは、`|` を間に入れて複数の型を指定できるようになった。

``` python
mask: list[bool | None]
info: dict[str, int | str]
data: int | float
```

### 型引数構文 ###

Python 3.12 からは、関数定義やクラス定義の構文が拡張され、具体的な型を指定する型ヒントではなく、さまざまな型に対応する汎用的な型ヒントを書くために型を引数として与えることができる。静的型付け言語において**ジェネリックス**（generics）と呼ばれる機能を型ヒントに導入した形になっている。

関数定義において、関数の名前に続いて `[]` で囲んで型引数を指定できる:

``` python
def func[T](a: list[T]) -> T:
    ...
```

この `T` は型引数であり、具体的な型を指定するのではなく、`a` の型を型引数 `T` に代入する、という指定になる。たとえば `func([1, 2, 3])` と呼び出した場合、引数 `a` には `list[int]` 型のオブジェクトを指定しているから、型引数 `T` は `int` になる（戻り値の型の指定は `int` になる）。

型引数 `T` に許容される型を指定するには、`T: int` のように `:` に続けて型を指定する。複数の型の中のいずれかであることを指定するときはタプルを使う:

``` python
def func[T: (int, str)](a: list[T]) -> T:
    ...
```

複数の型引数を要する場合は、次のように記述する。

``` python
def append_zero[T1, T2](a: tuple[T1, T2]) -> tuple[T1, T2, int]:
    """2要素タプルを受け取り、そのタプルの末尾に0を付加したタプルを返す関数"""
    return a + (0,)

if __name__ == "__main__":
    value = append_zero((1, "a"))  # valueの型は tuple[int, str, int]
```

任意のタプルの型定義も可能。そのためには、型引数の名前の先頭に `*` を付けて次のように記述する。

``` python
def variadic_append_zero[*Ts](a: tuple[*Ts]) -> tuple[*Ts, int]:
    """任意のタプルを受け取り、そのタプルの末尾に0を付加したタプルを返す関数"""
    return a + (0,)
```

たとえば `variadic_append_zero((1, "a"))` と呼び出した場合、引数 `a` の型は `tuple[int, str]` なので、戻り値の型 `tuple[*Ts, int]` は `tuple[int, str, int]` となる。

普通の型引数が型を代入する変数だとすると、`*` 付き型引数は複数の型を登録できる型のタプルに相当する。実際、`*` 付き型引数の代入はタプルのアンパックに似ている:

``` python
def variadic_get_middle[*Ts](a: tuple[int, *Ts, str]) -> tuple[*Ts]:
    """先頭が整数、末尾が文字列のタプルを受け取り、先頭と末尾の要素を削除したタプルを返す関数"""
    return a[1:-1]

if __name__ == "__main__":
    value = variadic_get_middle((0, "1", 2, "a"))  # valueの型は tuple[str, int]
    value2 = variadic_get_middle((0, "a"))         # value2の型は tuple[()]
```

`*` 付き型引数は、関数の可変長位置引数にも使用できる。また、`**` 付き型引数は、関数の可変長キーワード引数に使用できる。

``` python
def func[T: int, *Ts, **P](*args: *Ts, arg: Callable[P, T] = some_default):
    ...
```

クラス定義に型引数を指定することもできる。型引数を指定したクラス（**ジェネリッククラス**）を使用する場合、クラス名に `[]` を付けて型を指定する。

``` python
class C[T]:
    foo: T | None
    def __init__(self, foo: T | None = None):
        self.foo = foo

if __name__ == "__main__":
    c = C[str]("Hello")
```

Python 3.13 からは、型引数にデフォルトの型を指定できるようになった。

``` python
class C[T = int]:  # T のデフォルトを int に指定
    foo: T | None
    def __init__(self, foo: T | None = None):
        self.foo = foo

if __name__ == "__main__":
    c = C(10)  # C[int](10) と同じ
```

### 型エイリアスと type 文 ###

アノテーションは式として評価されるから、変数を使用できる。変数は「型アノテーションとして許容される形式」を代入する形で定義されていた場合、その型アノテーションの別名として使用される。この変数を**型エイリアス**（type alias）と呼ぶ。次のコードでは、型エイリアス `Vector` を定義する:

``` python
Vector = list[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]
```

しかし、この書き方には問題があって、静的型チェッカーは、1 行目が型エイリアスの定義なのか単なる代入文なのか判別できないので、エラーがあっても適切なメッセージを表示できない場合があった。そこで、Python 3.12 からは、型エイリアス定義のための新しい構文 type 文が導入された:

``` python
type Vector = list[float]
```
この `type` は、type 文の中でのみ識別子として予約されるソフトキーワードである。type 文以外の場面では `type` は普通の名前として使用でき、組み込みの名前が入った名前空間において `type` クラスオブジェクトの名前に使われている。

type 文でジェネリック型のエイリアスを作成することもできる:

``` python
type ListOrSet[T] = list[T] | set[T]
```

### 型推論の検証 ###

標準ライブラリの `typing` モジュールは、型ヒントに役立つツールを提供している。

Python 3.11 からは、以下の関数を使用できる。

``` python
typing.assert_type(val, typ, /)
```

この関数は、実行時には第 1 引数をそのまま返すだけで、何も起こらない。静的型チェッカーに対して、第 1 引数のオブジェクトが第 2 引数の型に推論されることを確認するように指示するために使用される。

静的型チェッカーは、`assert_type()` の呼び出しに遭遇した場合、第 1 引数のオブジェクトが第 2 引数の型とは異なる型であると推論したならばエラーを出力しなければならない。

``` python
typing.reveal_type(obj, /)
```

この関数は、実行時の引数の型を `sys.stderr` に出力し、引数をそのまま返す。静的型チェッカーに推論した型を問い合わせるために使用される。

静的型チェッカーはこの関数の呼び出しに遭遇した場合、推論した引数の型を示す診断メッセージを出力しなければならない。

ほとんどの静的型チェッカーは、`typing.reveal_type()` が提供される以前から独自に `reveal_type()` をサポートしていた。この構文は、Python の構文ではないので、消さずにコードを Python インタープリターで実行すると `NameError` 例外が発生する。名前を `typing` からインポートすることで、`NameError` 例外なしでコードを実行できる。

特殊型付けプリミティブ
----------------------

`typing` モジュールは、もっぱら型アノテーションの中で特殊な型や形式の表現に使われるようなクラスを提供している。

### Any ###

`typing.Any` は、制約のない型であることを示す特別な型である。型情報のないデータを取ってくる場合などに使う。

型引数構文で型を決定できない型引数は、`typing.Any` とされる。ただし、Python 3.13 からはデフォルトの型を指定した場合、型引数にはデフォルトの型が付けられる。

``` python
from typing import Any, assert_type

class C[T]:
    foo: T | None

    def __init__(self, foo: T | None = None):
        self.foo = foo

if __name__ == "__main__":
    c1 = C()
    assert_type(c1, C[Any])
    c2 = C[int]()
    assert_type(c2, C[int])
```

### Literal ###

`typing.Literal` は、指定された値と等しい値を持つことを示す型である。たとえば

``` python
from typing import Any, Literal

def validate_simple(data: Any) -> Literal[True]:  # 関数は必ず真を返す
    ...

type Mode = Literal['r', 'rb', 'w', 'wb']  # Mode は 'r', 'rb', 'w', 'wb' のいずれか1つの値をとる
def open_helper(file: str, mode: Mode) -> str:
    ...
```

### LiteralString ###

`typing.LiteralString` は、文字列定数を表す型である（Python 3.11 で追加）。ここで、文字列定数とはプログラムのソースコード中に直接書き込まれた文字列のことを指す。`typing.Literal` と異なり、`typing.LiteralString` は内容が指定されない。

``` python
from typing import LiteralString

def f(command: LiteralString):
    ...

f("DELETE ALL")  # 文字列定数なのでOK
f(input())  # 外部から入力した文字列なのでError
```

一般に、ファイルやネットワークなど、外部から取得した文字列は、式として評価したり文として実行したりすると、セキュリティ上の弱点となる。しかし、文字列定数はプログラムのソースコード中に直接書き込まれた文字列なので、外部からの攻撃に利用される可能性は低く、そのまま利用できる。`typing.LiteralString` は、このような外部から取得した可能性がある危険な文字列と、安全な文字列定数を区別する場合に利用される。

### Final ###

`typing.Final` は、再代入（再束縛）できないことを示す型である。Python では再代入できない変数を宣言できないが、変数の型アノテーションで `typing.Final` を使うと、その変数への再代入に対して静的型チェッカーはエラーを出力する。`typing.Final` はクラス変数に対しても有効で、サブクラスでオーバーライドすると静的型チェッカーはエラーを出力する。

In [ ]:
from typing import Final

MAX_SIZE: Final = 8192

def factory():
    ...
    global MAX_SIZE
    MAX_SIZE = 1  # Error reported by type checker

class Connection:
    TIMEOUT: Final[int] = 10

class FastConnector(Connection):
    TIMEOUT = 1  # Error reported by type checker

### TypeVar・ParamSpec・Generic ###

Python 3.12 で導入された型引数構文は、Python 3.11 で追加された `typing.TypeVar`, `typing.TypeVarTuple`, `typing.ParamSpec`, `typing.Generic` をより簡潔に書くための糖衣構文である。このため、これらは非推奨とされていないが、Python 3.12 で導入された型引数構文を使うべきであろう。

**ジェネリック関数**:

``` python
# Python 3.11 以前
from typing import TypeVar
_T = TypeVar("_T")
def func(a: list[_T]) -> _T:
    ...

# Python 3.12 以降
def func[T](a: list[T]) -> T:
    ...
```

**ジェネリックタプル**:

``` python
# Python 3.11 以前
from typing import TypeVar, TypeVarTuple
_T = TypeVar("_T")
_Ts = TypeVarTuple("_Ts")
def func(a: tuple[_T, *_Ts]) -> tuple[*_Ts, _T]:
    ...

# Python 3.12 以降
def func[T, *Ts](a: tuple[T, *Ts]) -> tuple[*Ts, T]:
    ...
```

**可変長引数**:

``` python
# Python 3.11 以前
from typing import TypeVar, ParamSpec
_P = ParamSpec("_P")
_R = TypeVar("_R")
def func(*args: _P.args, **kwargs: _P.kwargs) -> _R:
    ...

# Python 3.12 以降
def func[*Ts, **P, R](*args: *Ts, **kwargs: **P) -> R:
    ...
```

**ジェネリッククラス**:

``` python
# Python 3.11 以前
from typing import TypeVar, Generic
_T = TypeVar('_T')
class C(Generic[_T]):
    def meth(self) -> _T:
        ...

# Python 3.12 以降
class C[T]:
    def meth(self) -> T:
        ...
```

Python 3.13 からは、 `typing.TypeVar`, `typing.TypeVarTuple`, `typing.ParamSpec` のコンストラクタに `default` キーワード専用引数が追加され、デフォルトの型を指定できるようになった。型を決定できない場合は `typing.Any` とされるのであるが、デフォルトの型を指定した場合にはその型が付けられる。

``` python
from typing import TypeVar, Generic
_T = TypeVar("_T", default=int)  # _T のデフォルト値を int に指定
class C(Generic[_T]):
    foo: _T | None
    def __init__(self, foo: _T | None = None):
        self.foo = foo
```

### Self ###

クラス定義の中で、そのクラス自身の識別子（つまりクラス名）をアノテーションに記述することはできない。クラス定義を実行しないと、クラス名が束縛されないからである。ただし、アノテーションに文字列でクラス名を記述することで、静的型チェッカーに型チェックを行わせることができる。

次の `returns_eggs()` メソッドの定義では、メソッドから `Eggs` 以外の型のオブジェクトが返される場合、静的型チェッカーはエラーになる。

``` python
class Eggs:
    def returns_eggs(self) -> "Eggs":
        return Eggs()
```

Python 3.10 以前では、`typing.TypeVar` を使って型変数を定義すれば、文字列を使わずに定義中のクラス自身を表す型アノテーションを書くことができた。

``` python
from typing import TypeVar

Self = TypeVar("Self", bound="Spam")

class Spam:
    def return_me(self: Self) -> Self:
        return self
```

ただし、`typing.TypeVar` で定義した型変数はクラス変数の定義には使えなかった。

Python 3.11 で `typing.Self` が追加され、簡潔に定義中のクラス自身を表すことができるようになった。 `typing.Self` はクラス変数にも使用できる。

``` python
from typing import Self

class Spam:
    next_spam: Self | None = None
    def join_me(self, other: Self) -> Self:
        self.next_spam = other
        return self
```

`typing.Self` は動的に定義中のクラス自身を表す。クラスが継承されたときには、`typing.Self` は派生クラスを示すことになる。したがって、上記の `returns_eggs()` メソッドの定義に `typing.Self` を使うと間違った型ヒントになることに注意する。

### 非推奨のエイリアス ###

`typing` は、以下のような型のエイリアスも提供しているが、Python のバージョンアップにより使用が非推奨とされている。

  * `List[X]`, `Tuple[X]`, `Set[X]`, `FrozenSet[X]`, `Dict[X, Y]` → `list[X]`, `tuple[X]`, `set[X]`, `frozenset[X]`, `dict[X, Y]` を使うべき
  * `Type[X]` → `type[X]` を使うべき
  * `Union[X, Y]` → `X | Y` を使うべき
  * `Optional[X]` → `X | None` を使うべき
  * `Callable`、`Generator`、`Iterator`、`Iterable` → `collections.abc` が提供する抽象基底クラスを使うべき
  * `TypeAlias` → type 文を使うべき

``` python
# Python 3.8 以前
# 引数 cls はクラス MyClass そのもの（クラスオブジェクト）を取る（MyClass はインポートされていると仮定）
from typing import Type
def func(cls: Type[MyClass]) -> None:
    m = cls()
    m.meth()

# Python 3.9 以降
def func(cls: type[MyClass]) -> None:
    m = cls()
    m.meth()
```

``` python
# Python 3.11 以前
from typing import TypeAlias
Factors: TypeAlias = list[int]

# Python 3.12 以降
type Factors = list[int]
```

型ガード
--------

関数が異なる型の引数を受け取る場合、実行時のロジックによって変数の型が変わるので、静的型チェッカーだけでは型の絞り込みが難しい。このような状況で、動的な型チェックを行う文や関数により「ある条件を満たすと型が特定できる」ことを静的型チェッカーに伝えるための仕組みを**型ガード**（type guard）という。

### is None ###

変数が `None` を許容される場合に、if 文で `is None` をチェックして `None` の場合をはじくコードは自動的に型ガードとされる。つまり、静的型チェッカーにおいても変数の型が `None` ではない方の型と推論される。

たとえば、次のコードでは `message` 引数が `None` を許容されるが、if 文が型ガードとなるため、静的型チェッカーは 4 行目の `message` を `None` ではなく `str` 型と推論して `replace()` メソッドの参照についてエラーを出力しない。

``` python
def func(message: str | None) -> None:
    if message is None:
        return
    print(message.replace("you", "i"))
```

もし、このコードの if 文を削除すると、静的型チェッカーは「`None` は `replace` 属性を持たない」と指摘してエラーを出力する。

### is not None ###

変数が `None` を許容される場合に、if 文で `is not None` をチェックすると自動的に型ガードとされる。つまり、静的型チェッカーにおいても変数の型が `None` でない方の型と推論される。

たとえば、次のコードでは `message` 引数が `None` を許容されるが、if 文が型ガードとなるため、静的型チェッカーは 3 行目の `message` を `str` 型と推論して `replace()` メソッドの参照についてエラーを出力しない。

``` python
def func(message: str | None) -> None:
    if message is not None:
        print(message.replace("you", "i"))
```

もし、このコードの if 文によるチェックをしないと、静的型チェッカーは「`None` は `replace` 属性を持たない」と指摘してエラーを出力する。

### isinstance() ###

変数が複数の型を許容される場合に、`isinstance()` の結果をチェックするコードは自動的に型ガードとされる。

たとえば、次のコードでは `value` 引数が `int` 型と `str` 型を許容されるが、`isinstance()` の結果をチェックするコードが型ガードとなるため、静的型チェッカーは 3 行目の `value` を `int` 型と推論して `int` 型との足し算についてエラーを出力しない。

``` python
def func(value: int | str) -> None:
    if isinstance(value, int):
        print(value + 1)
```

もし、このコードの if 文によるチェックをしないと、静的型チェッカーは「型が判明しない `value` の値に int 型を足そうとしている」と指摘してエラーを出力する。

### Literal の値 ##

`Literal` で指定した複数の値のうち、どれと一致するかをチェックするコードは自動的に型ガードとされる。つまり、静的型チェッカーにおいても `Literal` を絞り込むことができる。

``` python
from typing import Literal

type Animal = Literal["cat", "dog", "pig"]

def func(pet: Animal) -> None:
    if pet == "cat":
        ...
```

このコードの if 文の中では、静的型チェッカーは変数 `pet` の値を `Literal["cat"]` 型と推論することができる。

### TypeGuard ###

動的に型を絞り込む関数を定義し、if 文でその関数の結果を使って条件分岐しても、自動的には型ガードとならない。静的型チェッカーは、ユーザー定義関数が型を絞り込む機能をサポートするかどうかわからないからである。

Python 3.10 からは、ユーザー定義関数の戻り値の型を `typing.TypeGuard[T]`（`T` は特定の型名）と指定することによって、その関数に入力して `True` が返ってきたときの変数は型が `T` であると静的型チェッカーに伝えることができる。

`typing.TypeGuard` を付ける関数（型ガード関数）は、ブール値を返すこと以外に制限はない。2 番目以降の引数を取って戻り値に利用してもよい。

このように型ガード関数の内容に踏み込まない代わりに、型推論は以下のルールが定められて限定的なものとされる。

  1. `typing.TypeGuard[T]` とした型ガード関数が `True` を返す場合、入力した変数の型は必ず `T` として扱う。それ以外の情報を型推論に使ってはならない。
  2. 型ガード関数が `False` を返す場合、その情報を型推論に使ってはならない。

たとえば、次のコードでは、`is_str_list()` 関数を型ガード関数として `func()` 関数の中で使用している。

In [1]:
from typing import Any, TypeGuard, assert_type

def is_str_list(val: list[Any]) -> TypeGuard[list[str]]:
    return all(isinstance(x, str) for x in val)

def func(list_val: list[str] | list[int]) -> None:
    if is_str_list(list_val):
        assert_type(list_val, list[str])
    else:
        assert_type(list_val, list[str] | list[int])

1 つ目の `assert_type()` では、`list[str]` 型と特定されることを検証する。

2 つ目の `assert_type()` では、型が絞り込まれないことを検証する。else のブロックで `list_val` が `list[int]` 型であることは明らかだが、ルール 2 により静的型チェッカーはそのような型推論を許されていない。

次のコードでは、入力の型がユーザー定義型 `A` の時に `True` を返す `is_A()` 関数を型ガード関数として `func()` 関数の中で使用している。

In [4]:
from typing import TypeGuard, assert_type

class A: ...
class B(A): ...
class C: ...

def is_A(obj: object) -> TypeGuard[A]:
    return isinstance(obj, A)

def func(obj: B | C) -> None:
    if is_A(obj):
        assert_type(obj, A)  # B 型には特定されない
        assert_type(obj, B)  # これはエラー

この 2 つの `assert_type()` では、`obj` が `A` 型と特定され `B` 型とは特定されないことを検証する。if ブロックの中で `obj` が `B` 型であることは明らかだが、ルール 1 により静的型チェッカーは形式的に `A` 型と特定する。

### TypeIs ###

型ガード関数は自由度が高く、型推論を改善することに限界がある。

Python 3.13 からは、`typing.TypeIs` が使えるようになった。`typing.TypeIs` は `typing.TypeGuard` と似ているが、`typing.TypeIs` を付ける関数（**型述語関数**; type predicate function）には以下の制限がある。

  1. 型述語関数は、ブール値を返す。
  2. 型述語関数は、引数の型と互換性を持つ型を絞り込むものでなければならない。
  3. 複数の引数を取るときは、2 番目以降の引数を型の絞り込みに利用してはならない。

型述語関数が内容的にも引数の型を絞り込む関数であるとの制限があるため、型推論のルールは型ガード関数の場合より緩和される。

先に示した `is_str_list()` 関数は、`list[Any]` 型の入力に対して互換性のある `list[str]` 型に絞り込むので型述語関数として使用できる。実際、`typing.TypeGuard` を `typing.TypeIs` に代えると、if ブロックと else ブロックで期待通りの型推論が行われる:

``` python
from typing import Any, TypeIs, assert_type

def is_str_list(val: list[Any]) -> TypeIs[list[str]]:
    return all(isinstance(x, str) for x in val)

def func(list_val: list[str] | list[int]) -> None:
    if is_str_list(list_val):
        assert_type(list_val, list[str])  # list[str] 型に絞り込む
    else:
        assert_type(list_val, list[int])  # list[int] 型に絞り込む
```

また、先に示した `is_A()` 関数を型述語関数として使用する場合、if ブロックで期待通りの型推論が行われる:

``` python
from typing import TypeIs, assert_type

class A: ...
class B(A): ...
class C: ...

def is_A(obj: object) -> TypeIs[A]:
    return isinstance(obj, A)

def func(obj: B | C) -> None:
    if is_A(obj):
        assert_type(obj, B)  # B 型に絞り込む
```

オーバーライドに関するデコレーター
----------------------------------

`typing` モジュールは、オーバーライドを安全に行うためのデコレーターを提供している。

あるクラスのメソッドを具体的に実装し、他のクラスでオーバーライドさせたくない場合、`typing.final()` デコレーターを使うと、そのメソッドのオーバーライドに対して静的型チェッカーはエラーを出力する。さらに、クラスの継承そのものを禁止したい場合には、`typing.final()` をクラスデコレーターとして使うと、継承に対して静的型チェッカーがエラーを出力する。

以下のコードでは、`typing.final()` を使って Colab 上（あるいは VSCode 上）でエラーが表示されるようになる。

In [ ]:
from typing import final

class Base:
    @final
    def done(self):
        ...

class Sub(Base):
    def done(self):  # Error reported by type checker
        ...

@final
class Leaf:
    ...

class Other(Leaf):  # Error reported by type checker
    ...

Python には言語レベルでオーバーライドやクラス継承を禁止する機能がないので、`typing.final()` デコレーターを使っても実行時に検査されないことに注意する。

また、Python 3.12 からは、`typing.override()` デコレーターを使って、メソッドが親クラスのメソッドをオーバーライドしていることを明示的に示すことができる。親クラスと整合性が取れないオーバーライドに対して、静的型チェッカーがエラーを出力するようになる。

``` python
from typing import override

class Base:
    def log_status(self):
        ...

class Sub(Base):
    @override
    def log_status(self):  # Okay: overrides Base.log_status
        ...

    @override
    def done(self):  # Error reported by type checker
        ...
```

オーバーロード
--------------

名前は同じだが仮引数の個数や型が違う関数を複数定義し、呼び出し時に実引数の個数や型に応じて選択することで複数の動作を行わせる仕組みを**オーバーロード**（overload）という。同名の関数のうちどれを呼び出すか探す処理を**オーバーロード解決**（overload resolution）と呼ぶ。

複数の型の許可や型エイリアスでは対応しきれないような複雑な引数と戻り値の組み合わせを持つ関数を、型付きで定義するにはオーバーロードを使う。ただし、Python は言語レベルでオーバーロードをサポートしていないので、同名の関数定義は引数の個数や型が違っていても常に上書きとなるのであって、オーバーロード解決は行われない。したがって、1 つの関数ですべての引数のパターンについて分岐処理を書かなければならない。

`typing.overload()` デコレーターは、分岐処理を書いた関数の前にある同名の関数定義を静的型チェッカーにオーバーロード関数と認識させるために付ける。オーバーロード解決は行われないから、このデコレーターを付ける関数には処理を書かない。また、このデコレーターを付けた関数を分岐処理を書いた関数より後ろに置かないこと。

In [ ]:
from typing import overload

@overload
def func() -> None: pass

@overload
def func(arg: str) -> None: pass

@overload
def func(arg: int) -> None: pass

def func(arg=None) -> None:
    """見せかけのオーバーロード関数"""
    if arg is None:
        print("arg is None")
    elif isinstance(arg, str):
        print("arg is string")
    elif isinstance(arg, int):
        print("arg is integer")
    else:
        raise TypeError

if __name__ == "__main__":
    func()
    func("hoge")
    func(1)

arg is None
arg is string
arg is integer


上記のコードは、Pylance を導入した VSCode 上なら `func()` を入力する時に引数と戻り値の全ての組み合わせが表示される。

mypy
----

[mypy](https://mypy-lang.org/) は、Python 公式の GitHub リポジトリで開発されている静的型チェッカーである。ライセンスは MIT license。インストール方法は次のとおり。

``` shell
pip install mypy
```

`mypy` は動作がかなり遅いので、主に CI に組み込んで使う。`mypy` を CI に組み込むときは、`mypy` のバージョンを指定してインストールするように構成する。`mypy` はバージョンアップに伴って頻繁に型チェックの仕様が変更されるからである。

### 使い方 ###

`mypy` の使い方は次のとおり:

``` shell
mypy [options] [directories or files ...]
```

ディレクトリを指定すると、サブディレクトリに対しても再帰的にチェックが行われることに注意する。

`mypy -h` または `mypy --help` でヘルプが画面に表示される。

`mypy` は静的型チェックを行うので、実行時にエラーとならない場合でも、型の整合性に問題があればエラーを表示する。`script.py` に次のコードを書いた場合:

``` python
def add(a: int, b: int) -> int:
    return a + b


if __name__ == "__main__":
    print(add("hoge", "fuga"))
```

`python script.py` を実行すると、文字列 `'hogefuga'` を出力し、エラーとならない。しかし、`mypy script.py` を実行すると、`add()` 呼び出しにおいて第 1 引数と第 2 引数の型が整合していないので、次のようなエラーが表示される。

``` shell
script.py:6: error: Argument 1 to "add" has incompatible type "str"; expected "int"  [arg-type]
script.py:6: error: Argument 2 to "add" has incompatible type "str"; expected "int"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)
```

### オプション ###

コマンドラインでオプションに

``` shell
--config-file CONFIG_FILE
```

を指定すると、指定したファイルからオプションの設定を読み取る。デフォルトでは、次の優先順位でファイルからオプションの設定を読み取る。

  1. `./mypy.ini`
  2. `./.mypy.ini`
  3. `./pyproject.toml`
  4. `./setup.cfg`
  5. `$XDG_CONFIG_HOME/mypy/config`
  6. `~/.config/mypy/config`
  7. `~/.mypy.ini`

`pyproject.toml` での設定ファイルの書き方は、次のようになる（[公式ドキュメント](https://mypy.readthedocs.io/en/stable/config_file.html#example-pyproject-toml)から引用）。

``` ini
# mypy global options:

[tool.mypy]
python_version = "2.7"
warn_return_any = true
warn_unused_configs = true
exclude = [
    '^file1\.py$',  # TOML literal string (single-quotes, no escaping necessary)
    "^file2\\.py$",  # TOML basic string (double-quotes, backslash and other characters need escaping)
]

# mypy per-module options:

[[tool.mypy.overrides]]
module = "mycode.foo.*"
disallow_untyped_defs = true

[[tool.mypy.overrides]]
module = "mycode.bar"
warn_return_any = false

[[tool.mypy.overrides]]
module = [
    "somelibrary",
    "some_other_library"
]
ignore_missing_imports = true
```

上記の設定のように、特定のモジュールでのみ有効なオプションを設定する場合、`[[tool.mypy.overrides]]` テーブルにモジュール名を指定する形で記述する。モジュール名にワイルドカード `*` を指定すると、特定のモジュール群に対してまとめて設定を書くことができる。また、複数のモジュールをリストで指定することもできる。

設定ファイルよりコマンドラインのオプション指定のほうが優先される。**設定ファイルとコマンドラインとでは、オプションの名前が多少異なる**ことに注意する。設定ファイルでのオプションの名前に対して、コマンドラインでのオプションの名前は、先頭に `--` を付け、アンダースコア `_` をハイフン `-` に変換したものになる。たとえば、設定ファイルの次の行

``` ini
disallow_any_generics = true
```

これはコマンドラインでオプション

``` shell
--disallow-any-generics
```

を指定するのと等価である。

以下では、オプションの名前は設定ファイル用のもので表す。コマンドラインで指定する場合は、オプションの名前を適切に変換すること。

設定ファイルで以下のオプションを `True` に指定すると（コマンドラインならオプションを追加すると）、デフォルトより厳格な型チェックを行う。

| オプション | 意味 | 逆のオプション |
|:---|:---|:---|
| `disallow_any_generics` | 型引数構文で `Any` の使用を禁止する | `allow_any_generics` |
| `disallow_subclassing_any` | `Any` 型のサブクラスの定義を禁止する | `allow_subclassing_any` |
| `check_untyped_defs` | 型ヒントのない関数でもその内部を型チェックする | `no_check_untyped_defs` |
| `disallow_untyped_calls` | 型ヒント付き関数から型ヒントなし関数を呼び出すことを禁止する | `allow_untyped_calls` |
| `disallow_untyped_decorators` | 型ヒント付き関数に型ヒントのないデコレーターを付けることを禁止する | `allow_untyped_decorators` |
| `disallow_incomplete_defs` | 型ヒントの不完全な関数を定義することを禁止するが、完全にアノテーションのない定義は許可<br />する | `allow_incomplete_defs` |
| `disallow_untyped_defs` |  型ヒントのない関数を定義すること、または不完全な型ヒントで関数を定義することを禁止する | `allow_untyped_defs` |
| `warn_redundant_casts` | `str('hoge')` のように式を推論された型に変換することについて警告する | `no_warn_redundant_casts` |
| `warn_unused_ignores` | 元々エラーとならない箇所の `# type: ignore` コメントについて警告する | `no_warn_unused_ignores` |
| `warn_return_any` | 戻り値が `Any` 型ではない関数で `Any` 型が返される場合に警告する | `no_warn_return_any` |
| `no_implicit_reexport` | パッケージの `foo/__init__.py` の中で `from .bar import bar` としている場合でも、プログラ<br />ムの中で `from foo import bar` とインポートすることを禁止する（`from foo.bar import bar` <br />とすべき） | `implicit_reexport` |
| `strict_equality` | `42 == 'hoge'` のような常に偽となる `==`, `is`, `in` を禁止する | `no_strict_equality` |
| `extra_checks` | 現在のコードでは問題がなくても、サブクラスが追加されたりすると型安全でなくなるような書<br />き方を禁止する | `no_extra_checks` |
| `warn_unused_configs` | 設定ファイルで `[mypy-<pattern>]` セクションを使用することについて警告する | `no_warn_unused_configs` |
| `strict` | Strict モード。すなわち、上の全てのオプションを有効にする | |

逆に、設定ファイルで以下のオプションを `True` に指定すると（コマンドラインならオプションを追加すると）、デフォルトより緩い型チェックを行う。

| オプション | 意味 | 逆のオプション |
|:---|:---|:---|
| `allow_redefinition` | 関数内なら同じ変数名で異なる型の再定義を許可する | `disallow_redefinition` |
| `no_strict_optional` | `None` を許容していない型の引数に `None` を渡してもエラーを抑制する | `strict_optional` |
| `ignore_missing_imports` | インポートしたモジュールに関するチェックを抑制する | |

また、次のオプションもよく使われる。

| オプション | 意味 |
|:---|:---|
| `python_version = X.Y`| Python X.Y 以前でサポートされていた機能でチェックする |
| `cache_dir = DIR` | キャッシュを保存するディレクトリを指定する。デフォルトでは、カレントディレクトリ直下の `.mypy_cache`。<br />キャッシュへの書き込みを無効にするには、`cache_dir = /dev/null`（UNIX）または `cache_dir = nul`<br />（Windows）とする |
| `follow_imports = {normal,silent,skip,error}` | インポートしたモジュールをチェックするルールを指定する。デフォルトは `normal` |
| `exclude = REGULAR_EXPRESSION` | 除外するファイルやディレクトリを正規表現で指定する |

なお、上記のように引数を伴うオプションをコマンドラインで指定する場合は、`--python-version X.Y` のように `=` を付けず空白を置いて引数を指定する。

オプションの詳細は、[公式ドキュメント](https://mypy.readthedocs.io/en/stable/command_line.html)を参照。

### スタブファイル ###

**スタブファイル**とは、ソースコードとは別に、拡張子 `.pyi` のファイルに型情報を記述したものをいう。`mypy` は、インポートするモジュールに型情報がなければ、スタブファイルから型情報を得ようとする。スタブファイルの中身は次のようになっており、関数の中身は `...` で省略できる。

``` python
def add(x: int, y: int) -> int: ...
```

サードパーティーのライブラリの一部は、型定義のリポジトリである [typeshed](https://github.com/python/typeshed) にスタブファイルが登録されている。

`mypy` のテスト結果が次のようなエラー

``` shell
error: Library stbus not installed for ライブラリ名
```

このようなエラーを表示した場合は、次のオプション

``` shell
--install-types
```

を付けて `mypy` のテストを実行すると、該当のライブラリの型定義がインストールされる。

Pylance
-------

Pylance は、Microsoft 製の VSCode 拡張機能である。ライセンスは CC-BY-4.0。入力支援機能の一部として型チェック機能を提供する。ただし、型チェック機能はデフォルトでは無効になっている。Pylance の設定を開き、Type Checking Mode（`python.analysis.typeCheckingMode`）を `off` から `standard` または `strict` に変更すると、型チェック機能が有効になる。

  * `basic`: 不明なクラス変数のチェック、関数呼び出しの省略不可能な引数のチェックを行うが、型チェックは行わない。
  * `standard`: 引数と戻り値に型アノテーションが付いた関数について型チェックを行う。
  * `strict`: 引数と戻り値に型アノテーションのない関数の定義を禁止する。